In [1]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import datetime

In [2]:
dframe = pd.read_excel('demo_dataset_new.xlsx')
dframe['Date'] = pd.to_datetime(dframe['Date'])
min_month = dframe['Date'].dt.to_period('M').min()
max_month = dframe['Date'].dt.to_period('M').max()

In [185]:
print(min_date, max_date)

2020-01 2023-03


In [3]:
fields_of_struct = []
for i in range(len(dframe)):
    try:
        fields_of_struct.append(dframe.iloc[i]['Fields Of Study'].split(','))
    except:
        fields_of_struct.append("lack of data")
dframe['field_list'] = fields_of_struct
dframe = dframe.explode('field_list')
dframe['field_list'] = dframe['field_list'].apply(lambda x: x.strip() if x != None else x)
dframe['field_list'] = dframe['field_list'].apply(lambda x: ''.join([i for i in x if i.isalpha() or i == " "]) if x != None else x)

####################
# drop the line whose field_list starts with "oratory"
dframe = dframe[dframe['field_list'] != 'oratory was in fact a way of establishing selfworth among Native Americans']
####################

dframe_field_group = dframe.groupby('field_list')
dframe_field_group.head()

,Title,Author Name,Date,Abstract,Paper Citation Count,Venue,Fields Of Study,Counts,rank,field_list
0,wav2vec 2.0: A Framework for Self-Supervised L...,"Alexei Baevski, Henry Zhou, Abdel-rahman Moham...",2020-06-20,We show for the first time that learning power...,1633,Neural Information Processing Systems,"['Computer Science', 'Engineering']",8,170,Computer Science
0,wav2vec 2.0: A Framework for Self-Supervised L...,"Alexei Baevski, Henry Zhou, Abdel-rahman Moham...",2020-06-20,We show for the first time that learning power...,1633,Neural Information Processing Systems,"['Computer Science', 'Engineering']",8,170,Engineering
1,Conformer: Convolution-augmented Transformer f...,"Anmol Gulati, James Qin, C. Chiu, Niki Parmar,...",2020-05-16,Recently Transformer and Convolution neural ne...,1099,Interspeech,"['Computer Science', 'Engineering']",19,135,Computer Science
1,Conformer: Convolution-augmented Transformer f...,"Anmol Gulati, James Qin, C. Chiu, Niki Parmar,...",2020-05-16,Recently Transformer and Convolution neural ne...,1099,Interspeech,"['Computer Science', 'Engineering']",19,135,Engineering
2,HiFi-GAN: Generative Adversarial Networks for ...,"Jungil Kong, Jaehyeon Kim, Jaekyoung Bae,",2020-10-12,Several recent work on speech synthesis have e...,505,Neural Information Processing Systems,"['Computer Science', 'Engineering']",27,284,Computer Science
...,...,...,...,...,...,...,...,...,...,...
5714,"Parts of Speech, Lexical Categories, and Word ...","J. Kornfilt,",2020-01-30,The term 鈥減art of speech鈥?is a traditional one...,Rustam Fayzullaev,",['1659074619'",'1659173480',26,30,
6162,Enhancement in Xerostomia Patient Salivary Lub...,"H. Wan, A. Vissink, P. K. Sharma,",2020-05-06,Oral lubrication mediated by mucin and protein...,12,Journal of dentistry research,"['Chemistry', 'Medicine']",22,126,Chemistry
6284,Russia鈥檚 Crony Capitalism. The Path from Marke...,"V. Rogova,",2020-04-20,of human freedom and equality as being achieva...,19,European Studies,['Economics'],28,110,Economics
7882,Bank Deposit Flows and Textual Sentiment: When...,"D. Anastasiou, Apostolos G. Katsafados,",2021-04-24,We investigate whether the so-called textual s...,6,SSRN Electronic Journal,['Economics'],9,478,Economics


In [5]:
field_genres = []
field_counts = []
field_months = []

for field in dframe_field_group:
    for month in pd.date_range(min_month.to_timestamp(), max_month.to_timestamp(), freq='M'):
        field_genres.append(field[0])
        field_counts.append(len(field[1][field[1]['Date'].dt.to_period('M') == month.to_period('M')]))
        field_months.append(month)

field_df = pd.DataFrame({'fields': field_genres, 'counts': field_counts, 'months': field_months})

field_trace_dict = {}
field_all_traces = field_df['fields'].unique()
total_traces = len(field_all_traces)

for field in field_all_traces:
    field_trace_dict[field] = go.Scatter(x=field_df[field_df['fields'] == field]['months'], 
                                         y=field_df[field_df['fields'] == field]['counts'], 
                                         name=field,
                                         mode='none',
                                         stackgroup= total_traces,
                                         line_shape='spline')

In [7]:
fig = go.Figure()
for field in field_trace_dict.keys():
    fig.add_trace(field_trace_dict[field])

fig.update_layout(
    xaxis=dict(
        # rangeslider=dict(
        #     visible=True
        # ),
        # type="date"
    )
)

fig.show()